In [1]:
import sqlite3
from sqlite3 import Error

In [2]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time
import csv
from string import ascii_lowercase

In [3]:
import urllib.request

In [16]:
import random

## Tomato cultivar object that parses infos from html

In [4]:
class CultivarFetcher():
    
    def __init__(self, cultivar, url):
        #cultivar e.g. 'http://www.tomaten-atlas.de/sorten/a/2541-anderson-roman'
        #url: "http://www.tomaten-atlas.de"
        self.cultivar = self.get_url_content(cultivar)
        self.cultivar_name = self.cultivar.find("h2").text.strip().replace(" ","_").replace("/","_").replace(",","")
        self.url = url
        
    #def add(self, cultivar):
    #    self.cultivar = get_url_content(cultivar)
    
    def get_url_content(self, url):
        #general function to get the html page
        r = requests.get(url)
        doc = BeautifulSoup(r.text, "html.parser")
        return doc
    
    def parse_table(self):
        #function to get the characteristics for one cultivar including growth height, fruit size and color...
        #cultivar is the BeautifulSoup object
        characteristics = {}
        table = self.cultivar.find("div", {"id": "content"}).find("table")
        for i in table.find_all("tr"):
            c = i.find_all("td")
            characteristics[c[0].text.strip(":").replace(" ","_")] = c[1].text
        return characteristics
    
    def download_images(self):
        #cultivar is the BeautifulSoup object
        image_list = self.cultivar.find("div", {"id": "content"}).find("ul", {"class": "nogallery"})
        images = []
        for i in image_list.find_all("a"):
            if "id=0" not in i["href"]:
                url = self.url  + i["href"] #"http://www.tomaten-atlas.de"
                img = urllib.request.urlretrieve(url, '{}_{}.jpg'.format(
                            self.cultivar_name,
                            (i['title']
                                 # removing unwanted characters
                                .lower()
                                .replace(' ', '_')
                                .replace('.', '')
                                .replace(',', '')
                                .replace(':', '')
                                .replace('\'', '')
                                .replace('\/', '')
                                .replace('’', '')
                                .replace('#', '')
                                .replace('*', ''))
                        ))
                images.append(img[0]) #list images to store in db
            else:
                images.append("")
        return images
    
    def alt_names(self):
        #cultivar is the BeautifulSoup object
        alt = self.cultivar.find("div", {"id": "content"}).find_all("p")[0]
        alt_name = alt.text.split(":")[1].strip().split("\r\n")
        if "keine" in alt_name[0]:
            return None
        return ", ".join(alt_name)
    
    def get_origin(self):
        #cultivar is the BeautifulSoup object
        orig = self.cultivar.find("div", {"id": "content"}).find_all("p")[1]
        orig = orig.text.split(":")[1].strip().split("\r\n")
        return orig[0]
    
    def alt_img(self):
        #cultivar is the BeautifulSoup object
        #function to get additional images if available
        cultivar_name = self.cultivar.find("h2").text.strip()
        cultivar_name = cultivar_name.replace(" ","_")
        image_list = self.cultivar.find("div", {"id": "content"}).find_all("ul", {"class": "nogallery"})
        images = []
        if len(image_list) >1:
            for i in image_list[1].find_all("a"):
                url = self.url + i["href"]
                img = urllib.request.urlretrieve(url, '{}_{}.jpg'.format(
                            cultivar_name,
                            (i['href'].split("=")[1]
                                 # removing unwanted characters
                                .lower()
                                .replace(' ', '_')
                                .replace('.', '')
                                .replace(',', '')
                                .replace(':', '')
                                .replace('\'', '')
                                .replace('\/', '')
                                .replace('’', '')
                                .replace('#', '')
                                .replace('*', ''))
                        ))
                images.append(img[0]) #list images to store in db
            return images
        else:
            return None

## Class to create a Sqlite Database and to take care of tables and inserts

In [5]:
class SqliteDatabase():
    
    def __init__(self):
        self.connection = None
        self.tables = {}
    
    def create_connection(self, db):
        connection = None
        try:
            connection = sqlite3.connect(db)
        except Error as e:
            print(e)
        return connection
    
    def add_connection(self, db):
        self.connection = self.create_connection(db)
    
    def create_table(self, table_name, columns, types, constraints, foreign_key = None):
        #create_sql has to be a CREATE TABLE statement
        #columns: column names
        #types: data types same order as columns
        #constraints: same order as columns, if a column has no contraints: empty string
        #foreign_key: tuple with columnname and tablename and table_columnname
        try:
            cols = [col + " " + t + " " + con for col,t,con in list(zip(columns, types, constraints))]
            column = ", ".join(cols)
            if foreign_key == None:
                create_sql = "CREATE TABLE IF NOT EXISTS " + table_name + "("+ column + ");"
            else:
                create_sql = "CREATE TABLE IF NOT EXISTS " + table_name + "("+ column + ", " 
                create_sql += "FOREIGN KEY (" + foreign_key[0] +") REFERENCES "+ foreign_key[1]
                create_sql += " (" + foreign_key[2] + "));"
                print(create_sql)
            conn = self.connection.cursor()
            conn.execute(create_sql)
            self.tables[table_name] = columns
        except Error as e:
            print(e)

            
    def insert(self, table, values):
        #table: which table to insert
        #values: dict with columnname and value
        columns = ",".join(self.tables[table])
        cols = ""
        vals = []
        #print(values)
        for key, value in values.items():
            if key not in columns:
                print("There is no column "+ key)
                break
            else:
                cols += key + ","
                vals.append(value)
        cols = cols.rstrip(",")
        ins = ["?"] * len(vals)
        sql = "INSERT INTO " + table +"(" + cols + ") VALUES("+ ",".join(ins) +");"
        cur = self.connection.cursor()
        cur.execute(sql, vals)
        self.connection.commit()
        return cur.lastrowid

### create database instance and set up tables

In [6]:
tomatoes = SqliteDatabase()
tomatoes.add_connection("tomatoes.db")

In [7]:

cols = ["id", "Name", "Alt_Name", "Herkunft", "Fruchtform", "Farbe", "Fruchtgroesse", "Fruchtreife",
        "Fruchtkammern", "Schale", "Fleischigkeit", "Rippung", "Fruchtspitze", "gruene_Frucht", "Geschmack",
        "Fruchtstaende", "Wuchshoehe", "Blattform", "Bluete", "img_gruene_Frucht", "img_Frucht", 
        "img_geschnittene_Frucht"]
types = ["INTEGER", "text", "text", "text", "text", "text", "text", "text", "text", "text", "text", "text", "text", 
         "text", "text", "text", "text", "text", "text", "text", "text", "text"]
constr = ["PRIMARY KEY AUTOINCREMENT", "NOT NULL", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", ""]
tomatoes.create_table("cultivars", cols, types, constr)

In [8]:
cols = ["id", "img", "cultivar_id"]
types = ["INTEGER", "text", "INTEGER"]
constr = ["PRIMARY KEY AUTOINCREMENT", "NOT NULL",""]
foreign = ("cultivar_id", "cultivars", "id")
tomatoes.create_table("additional_images", cols, types, constr, foreign)

CREATE TABLE IF NOT EXISTS additional_images(id INTEGER PRIMARY KEY AUTOINCREMENT, img text NOT NULL, cultivar_id INTEGER , FOREIGN KEY (cultivar_id) REFERENCES cultivars (id));


### here we get the list with the links to all cultivars

In [9]:
def get_url_content(url):
        #general function to get the html page
        r = requests.get(url)
        doc = BeautifulSoup(r.text, "html.parser")
        return doc
    
def list_links(url):
    #to get the links to the cultivars shown in the alphabetical list
    cultivars = []
    doc = get_url_content(url)
    for i in doc.find("div", {"id": "content"}).find_all("a"):
        cultivars.append(i["href"])
    return cultivars

In [38]:
#here all the links to all cultivars are listed
url = "http://www.tomaten-atlas.de/sorten/"
alphabet = ascii_lowercase + "0"
cultivars = []
for c in alphabet:
    time.sleep(2)
    url_site = url + c
    cultivars += list_links(url_site)
#print(cultivars)

### iterate over links, create cultivar object and store in database

In [40]:
for c in range(7009,len(cultivars)):
    time.sleep(random.randint(0,10))
    cult = CultivarFetcher(cultivars[c], "http://www.tomaten-atlas.de")
    characteristics = cult.parse_table()
    images = cult.download_images()
    name = cult.alt_names()
    origin = cult.get_origin()
    alt_img = cult.alt_img()
    values = {"Name": cult.cultivar_name, "Alt_Name": name, "Herkunft": origin, 
              "Fruchtform":characteristics["Fruchtform"], "Farbe": characteristics['Farbe'],
              "Fruchtgroesse": characteristics['Fruchtgröße'],
              "Fruchtreife": characteristics['Fruchtreife'],
              "Fruchtkammern": characteristics['Fruchtkammern'],
              "Schale": characteristics['Schale'],
              "Fleischigkeit": characteristics['Fleischigkeit'], 
              "Rippung": characteristics['Rippung'],
              "Fruchtspitze": characteristics['Fruchtspitze'],
              "gruene_Frucht": characteristics['Grüne_Frucht'],
              "Geschmack": characteristics['Geschmack'],
              "Fruchtstaende": characteristics['Fruchtstände'],
              "Wuchshoehe": characteristics['Wuchshöhe'],
              "Blattform": characteristics['Blätterform'],
              "Bluete": images[0],
              "img_gruene_Frucht": images[1],
              "img_Frucht":images[2],
              "img_geschnittene_Frucht": images[3]}

    cult_id = tomatoes.insert("cultivars", values)
    if alt_img:
        for i in alt_img:
            v = {"img": i, "cultivar_id": cult_id}
            tomatoes.insert("additional_images", v)

In [41]:
cult_id

7028

In [39]:
cultivars[7009]

'http://www.tomaten-atlas.de/sorten/0/4826-0-33'